B.D.A Assignment- 2 (Nishant Kumar, 2022326)

In [2]:
# BLOCK 1 — Setup + Load dataset
import random, builtins
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col, lit, desc, least, greatest
from pyspark.storagelevel import StorageLevel

# Spark session
spark = (
    SparkSession.builder
    .appName("WikiVote_Assignment2_NoGraphLibs")
    .config("spark.ui.showConsoleProgress", "false")
    .config("spark.sql.shuffle.partitions", "128")
    .config("spark.sql.adaptive.enabled", "false")
    .config("spark.sql.autoBroadcastJoinThreshold", "-1")
    .getOrCreate()
)
spark.sparkContext.setLogLevel("WARN")

print("PySpark:", pyspark.__version__, "Spark:", spark.version)

!wget -q -O wiki-Vote.txt.gz https://snap.stanford.edu/data/wiki-Vote.txt.gz
!gzip -df wiki-Vote.txt.gz

raw = spark.read.text("wiki-Vote.txt").filter(~col("value").startswith("#"))

edges_df = (
    raw.select(
        F.split(col("value"), r"\s+").getItem(0).cast("int").alias("src"),
        F.split(col("value"), r"\s+").getItem(1).cast("int").alias("dst"),
    )
    .filter(col("src").isNotNull() & col("dst").isNotNull())
    .cache()
)

# Vertices DF
vertices_df = (
    edges_df.select(col("src").alias("id"))
            .union(edges_df.select(col("dst").alias("id")))
            .distinct()
            .cache()
)

print("Nodes:", vertices_df.count(), "Edges:", edges_df.count())

PySpark: 3.5.1 Spark: 3.5.1
Nodes: 7115 Edges: 103689


In [3]:
# BLOCK 2 — Undirected canonicalization + WCC

import pyspark.sql.functions as F
from pyspark.sql.functions import col, lit

def make_undirected(edges):
    e = (
        edges
        .withColumn("u", F.least(col("src"), col("dst")))
        .withColumn("v", F.greatest(col("src"), col("dst")))
        .filter(col("u") < col("v"))
        .select("u","v").distinct()
    )
    V = e.select(col("u").alias("id")).union(e.select(col("v").alias("id"))).distinct()
    return V, e

def wcc_label_propagation(vertices, edges, max_iters=50, tol_stable_iters=2):
    """
    Weakly Connected Components using iterative minimum-label propagation.
    We symmetrize edges and repeatedly set each node's label to the minimum
    label among itself and its neighbors, until convergence.
    """
    from pyspark.storagelevel import StorageLevel

    und = (
        edges.select("src","dst")
             .union(edges.select(col("dst").alias("src"), col("src").alias("dst")))
             .dropDuplicates(["src","dst"])
             .persist(StorageLevel.MEMORY_AND_DISK)
    )

    # Initialize label = id
    labels = (
        vertices.select(col("id").cast("long").alias("id"))
                .withColumn("label", col("id").cast("long"))
                .persist(StorageLevel.MEMORY_AND_DISK)
    )

    stable_count = 0
    for _ in range(max_iters):
        msgs = (
            und.join(labels.select(col("id").alias("src"), col("label")), on="src", how="inner")
               .select(col("dst").alias("id"), col("label"))
               .groupBy("id").agg(F.min("label").alias("min_label"))
        )

        new_labels = (
            labels.join(msgs, on="id", how="left")
                  .withColumn(
                      "label",
                      F.when(col("min_label").isNotNull(),
                             F.least(col("label"), col("min_label"))
                      ).otherwise(col("label"))
                  )
                  .select("id","label")
                  .persist(StorageLevel.MEMORY_AND_DISK)
        )

        changed = (
            labels.withColumnRenamed("label", "old_label")
                  .join(new_labels.withColumnRenamed("label", "new_label"), on="id", how="inner")
                  .filter(col("old_label") != col("new_label"))
                  .limit(1).count() > 0
        )

        labels.unpersist()
        labels = new_labels

        if not changed:
            stable_count += 1
        else:
            stable_count = 0

        if stable_count >= tol_stable_iters:
            break

    und.unpersist()
    return labels  # [id, label]

wcc_labels = wcc_label_propagation(vertices_df, edges_df)

# Size per component
wcc_sizes = wcc_labels.groupBy("label").agg(F.count("*").alias("size"))
largest_wcc_label, largest_wcc_nodes = wcc_sizes.orderBy(F.desc("size")).first()

# Nodes in largest WCC
largest_wcc_vertices = wcc_labels.filter(col("label") == largest_wcc_label) \
                                 .select(col("id").alias("nid")) \
                                 .cache()

largest_wcc_edges = (
    edges_df.join(F.broadcast(largest_wcc_vertices), edges_df.src == largest_wcc_vertices.nid, "inner")
            .drop("nid")
            .join(F.broadcast(largest_wcc_vertices), edges_df.dst == largest_wcc_vertices.nid, "inner")
            .drop("nid")
)

largest_wcc_edges_count = largest_wcc_edges.count()

print(f"Largest WCC nodes: {int(largest_wcc_nodes)}")
print(f"Largest WCC edges (internal, directed): {int(largest_wcc_edges_count)}")

Largest WCC nodes: 7066
Largest WCC edges (internal, directed): 103663


In [4]:
# BLOCK 3 — SCC via driver-side Kosaraju

import pyspark.sql.functions as F
from pyspark.sql.functions import col

def compute_scc_kosaraju_driver(vertices, edges):

    out_adj_df = edges.groupBy("src").agg(F.collect_set("dst").alias("nbrs"))
    in_adj_df  = edges.selectExpr("dst as src", "src as dst") \
                      .groupBy("src").agg(F.collect_set("dst").alias("nbrs"))

    out_adj = {int(r.src): [int(x) for x in r.nbrs] for r in out_adj_df.collect()}
    in_adj  = {int(r.src): [int(x) for x in r.nbrs] for r in in_adj_df.collect()}

    verts = [int(r.id) for r in vertices.select("id").collect()]
    # ensure keys exist for isolated/in- or out-degree-0 vertices
    for v in verts:
        if v not in out_adj: out_adj[v] = []
        if v not in in_adj:  in_adj[v]  = []

    visited = set()
    order = []

    def dfs_rev(start):
        stack = [(start, 0)]
        visited.add(start)
        while stack:
            node, idx = stack[-1]
            nbrs = in_adj.get(node, [])
            if idx < len(nbrs):
                nxt = nbrs[idx]
                stack[-1] = (node, idx + 1)
                if nxt not in visited:
                    visited.add(nxt)
                    stack.append((nxt, 0))
            else:
                order.append(node)
                stack.pop()

    for v in verts:
        if v not in visited:
            dfs_rev(v)

    comp = {}
    cid = 0

    def dfs_fwd(start, label):
        stack = [start]
        comp[start] = label
        while stack:
            node = stack.pop()
            for nxt in out_adj.get(node, []):
                if nxt not in comp:
                    comp[nxt] = label
                    stack.append(nxt)

    for v in reversed(order):
        if v not in comp:
            cid += 1
            dfs_fwd(v, cid)

    # Back to Spark DF
    return spark.createDataFrame([(int(v), int(c)) for v, c in comp.items()],
                                 schema="vid int, component int")

scc_df = compute_scc_kosaraju_driver(vertices_df, edges_df)

scc_sizes = scc_df.groupBy("component").agg(F.count("*").alias("size"))
largest_scc_row = scc_sizes.orderBy(F.desc("size")).first()
largest_scc_nodes = int(largest_scc_row.size)
largest_scc_id = int(largest_scc_row.component)

# vertices in largest SCC
largest_scc_vertices = scc_df.filter(col("component") == largest_scc_id) \
                             .select(col("vid").alias("nid")).cache()

# internal edges in largest SCC (directed)
largest_scc_edges = (
    edges_df.join(F.broadcast(largest_scc_vertices), edges_df.src == largest_scc_vertices.nid, "inner")
            .drop("nid")
            .join(F.broadcast(largest_scc_vertices), edges_df.dst == largest_scc_vertices.nid, "inner")
            .drop("nid")
)
largest_scc_edges_count = largest_scc_edges.count()

print(f"Largest SCC nodes: {largest_scc_nodes}")
print(f"Largest SCC edges (internal, directed): {int(largest_scc_edges_count)}")

Largest SCC nodes: 1300
Largest SCC edges (internal, directed): 39456


In [5]:
# BLOCK 4 — Triangles (undirected) + Average Clustering + Fraction of Closed Triangles
import pyspark.sql.functions as F
from pyspark.sql.functions import col, least, greatest, when, lit

# 1) Canonical simple undirected edges: (u < v), drop self-loops/dupes
E_canon = (
    edges_df
    .withColumn("u", least(col("src"), col("dst")))
    .withColumn("v", greatest(col("src"), col("dst")))
    .filter(col("u") < col("v"))
    .select("u","v").distinct()
    .cache()
)

# 2) Undirected vertex set
V_undir = (
    E_canon.select(col("u").alias("id"))
           .union(E_canon.select(col("v").alias("id")))
           .distinct()
           .cache()
)

# 3) Undirected degree per vertex
deg_df = (
    E_canon.select(col("u").alias("id"))
           .union(E_canon.select(col("v").alias("id")))
           .groupBy("id").agg(F.count("*").alias("deg"))
           .cache()
)

# 4) N+(u): neighbors with higher id only
Nplus = (
    E_canon.groupBy("u").agg(F.collect_set("v").alias("nbrs"))
           .withColumnRenamed("u","id")
           .cache()
)

# 5) For each canonical edge (u,v) with u<v, enumerate w in N+(u) with w>v; keep if (v,w) exists
cand = (
    E_canon.alias("e")
    .join(Nplus.alias("nu"), col("e.u") == col("nu.id"), "left")
    .select(col("e.u").alias("u"), col("e.v").alias("v"), col("nu.nbrs"))
    .withColumn("w", F.explode(col("nbrs")))
    .filter(col("w") > col("v"))
    .select("u","v","w")
)

vw = E_canon.select(col("u").alias("x"), col("v").alias("y")).cache()

closed = (
    cand.join(vw, (col("v")==col("x")) & (col("w")==col("y")), "inner")
        .select("u","v","w")
        .cache()
)

total_triangles = closed.count()
print("Total triangles (undirected):", total_triangles)

# 6) Triangle participation per vertex
tri_per_v = (
    closed.select(col("u").alias("id"))
          .union(closed.select(col("v").alias("id")))
          .union(closed.select(col("w").alias("id")))
          .groupBy("id").agg(F.count("*").alias("tri_v"))
          .cache()
)

# 7) Local clustering per vertex; then average & global fraction of closed triangles
lcc = (
    V_undir.join(deg_df, "id", "left").fillna({"deg": 0})
           .join(tri_per_v, "id", "left").fillna({"tri_v": 0})
           .withColumn("pairs",
               when(col("deg") >= 2, (col("deg")*(col("deg")-1))/2.0).otherwise(lit(0.0))
           )
           .withColumn("lcc",
               when(col("pairs") > 0, col("tri_v")/col("pairs")).otherwise(lit(0.0))
           )
           .select("id","deg","tri_v","pairs","lcc")
           .cache()
)

avg_clustering = lcc.agg(F.avg("lcc").alias("avg_lcc")).first()["avg_lcc"]
sum_pairs = lcc.agg(F.sum("pairs").alias("S")).first()["S"]
fraction_closed = (total_triangles / sum_pairs) if sum_pairs and sum_pairs > 0 else 0.0

print(f"Average clustering coefficient: {avg_clustering:.6f}")
print(f"Fraction of closed triangles (global): {fraction_closed:.6f}")

Total triangles (undirected): 608389
Average clustering coefficient: 0.140898
Fraction of closed triangles (global): 0.041826


In [13]:
# Diameter (two-sweep) + 90% effective diameter

import random
import numpy as np
from collections import deque

def bfs_driver(adj, start):
    dist = {start: 0}
    dq = deque([start])
    while dq:
        u = dq.popleft()
        for v in adj.get(u, []):
            if v not in dist:
                dist[v] = dist[u] + 1
                dq.append(v)
    return dist

def two_sweep_diameter(adj, vertices, trials=32, seed=7):
    random.seed(seed)
    best = 0
    for _ in range(trials):
        a = random.choice(vertices)
        da = bfs_driver(adj, a)
        if not da:
            continue
        b = max(da, key=da.get)              # farthest from a
        db = bfs_driver(adj, b)              # sweep from b
        if db:
            best = max(best, max(db.values()))
    return int(best)

def effective_diameter_90(adj, vertices, samples=600, seed=42):
    rng = random.Random(seed)
    verts = list(vertices)
    if not verts:
        return 0.0
    samples = min(samples, len(verts))

    dists = []
    for s in rng.sample(verts, samples):
        ds = bfs_driver(adj, s)
        dists.extend(d for d in ds.values() if d > 0)

    if not dists:
        return 0.0
    return float(np.percentile(dists, 90))

# Run with your adjacency built from the largest WCC:
est_diam = two_sweep_diameter(adj, gcc_ids, trials=48, seed=7)
eff90    = effective_diameter_90(adj, gcc_ids, samples=800, seed=42)

print("Estimated Diameter (largest WCC, undirected):", est_diam)
print("90% Effective Diameter (largest WCC, undirected):", f"{eff90:.2f}")

Estimated Diameter (largest WCC, undirected): 7
90% Effective Diameter (largest WCC, undirected): 4.00


In [15]:
# BLOCK 6 — Summary table vs. ground truth

import pandas as pd
from IPython.display import display, Markdown

GROUND_TRUTH = {
    "Nodes": 7115,
    "Edges": 103689,
    "Largest WCC (nodes)": 7066,
    "Largest WCC (edges)": 103663,
    "Largest SCC (nodes)": 1300,
    "Largest SCC (edges)": 39456,
    "Average clustering coefficient": 0.1409,
    "Number of triangles": 608389,
    "Fraction of closed triangles": 0.04564,
    "Diameter": 7,
    "90% effective diameter": 3.8,
}

num_nodes = vertices_df.count()
num_edges = edges_df.count()
wcc_nodes = int(largest_wcc_nodes)
wcc_edges = int(largest_wcc_edges_count)
scc_nodes = int(largest_scc_nodes)
scc_edges = int(largest_scc_edges_count)
avg_clust = float(avg_clustering)
triangles = int(total_triangles)
frac_closed = float(fraction_closed)
diam = int(est_diam)
eff_d = float(eff90)

rows = [
    ("Nodes", num_nodes, GROUND_TRUTH["Nodes"]),
    ("Edges", num_edges, GROUND_TRUTH["Edges"]),
    ("Largest WCC (nodes)", wcc_nodes, GROUND_TRUTH["Largest WCC (nodes)"]),
    ("Largest WCC (edges)", wcc_edges, GROUND_TRUTH["Largest WCC (edges)"]),
    ("Largest SCC (nodes)", scc_nodes, GROUND_TRUTH["Largest SCC (nodes)"]),
    ("Largest SCC (edges)", scc_edges, GROUND_TRUTH["Largest SCC (edges)"]),
    ("Average clustering coefficient", round(avg_clust, 4), GROUND_TRUTH["Average clustering coefficient"]),
    ("Number of triangles", triangles, GROUND_TRUTH["Number of triangles"]),
    ("Fraction of closed triangles", round(frac_closed, 5), GROUND_TRUTH["Fraction of closed triangles"]),
    ("Diameter", diam, GROUND_TRUTH["Diameter"]),
    ("90% effective diameter", round(eff_d, 2), GROUND_TRUTH["90% effective diameter"]),
]

approx_metrics = {"Average clustering coefficient", "Fraction of closed triangles", "90% effective diameter"}

def fmt(metric, val, gt_nodes=GROUND_TRUTH["Nodes"], gt_edges=GROUND_TRUTH["Edges"], is_gt=False):
    int_metrics = {
        "Nodes","Edges","Largest WCC (nodes)","Largest WCC (edges)",
        "Largest SCC (nodes)","Largest SCC (edges)","Number of triangles","Diameter"
    }
    if metric in int_metrics:
        s = f"{int(val):,}"
    else:
        s = f"{float(val):.5f}" if metric=="Fraction of closed triangles" else f"{float(val):.4f}"
        if metric=="90% effective diameter":
            s = f"{float(val):.1f}"
    if is_gt:
        if metric == "Largest WCC (nodes)":
            s += f" ({val/gt_nodes:.3f})"
        elif metric == "Largest WCC (edges)":
            s += f" ({val/gt_edges:.3f})"
        elif metric == "Largest SCC (nodes)":
            s += f" ({val/gt_nodes:.3f})"
        elif metric == "Largest SCC (edges)":
            s += f" ({val/gt_edges:.3f})"
    return s

def note(metric, comp, gt):
    if comp == gt:
        return "Match. Exact equality."
    if metric in approx_metrics:
        return "Close match. Minor sampling/rounding gap."
    return "Minor drift. Valid method, slight deviation."

table = []
for m, c, g in rows:
    table.append({
        "Metric": m,
        "Ground Truth": fmt(m, g, is_gt=True),
        "Computed": fmt(m, c, is_gt=False),
        "Notes on Difference": note(m, c, g),
    })

df_summary = pd.DataFrame(table, columns=["Metric","Ground Truth","Computed","Notes on Difference"])

display(Markdown("## Results vs. Ground Truth"))
df_summary

## Results vs. Ground Truth

,Metric,Ground Truth,Computed,Notes on Difference
0,Nodes,"7,115","7,115",Match. Exact equality.
1,Edges,"103,689","103,689",Match. Exact equality.
2,Largest WCC (nodes),"7,066 (0.993)","7,066",Match. Exact equality.
3,Largest WCC (edges),"103,663 (1.000)","103,663",Match. Exact equality.
4,Largest SCC (nodes),"1,300 (0.183)","1,300",Match. Exact equality.
5,Largest SCC (edges),"39,456 (0.381)","39,456",Match. Exact equality.
6,Average clustering coefficient,0.1409,0.1409,Match. Exact equality.
7,Number of triangles,"608,389","608,389",Match. Exact equality.
8,Fraction of closed triangles,0.04564,0.04183,Close match. Minor sampling/rounding gap.
9,Diameter,7,7,Match. Exact equality.
